In [1]:
from google.colab import drive
drive.mount('/drive/')
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [65.5 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.down

In [2]:
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
import torch

okt = Okt()

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [8]:
!ls "/drive/My Drive"

'CNN_실습_(문제).ipynb'
'Colab Notebooks'
 data_deep.pkl
 data_ml.pkl
'아르바이트 시간과 학점의 연관성 조.gform'
'장학생 신청서 및 개인정보 수집이용 동의서 동의서.hwp'
'시각화 코드 절차.ipynb'
'Ques)Training_Neural_Network_.ipynb'
 sentiment_analysis.ipynb
'submission.csv의 사본'


https://github.com/rudvlf0413/skt-exercise/blob/master/sentiment_analysis/preprocessing.ipynb

In [0]:
import pickle

In [0]:
with open('/drive/My Drive/data_deep.pkl', 'rb') as f:
    index2voca = pickle.load(f)
    voca2index = pickle.load(f)
    train_X = pickle.load(f)
    train_y = pickle.load(f)
    test_X = pickle.load(f)
    test_y = pickle.load(f)

In [25]:
train_X.shape

(251880, 100)

In [0]:
train_X, train_y = torch.LongTensor(train_X), torch.FloatTensor(train_y)
test_X, test_y = torch.LongTensor(test_X), torch.FloatTensor(test_y)

train_y, test_y = train_y.view(-1, 1), test_y.view(-1, 1)

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

In [0]:
class CNNRegressor(nn.Module):
    
    def __init__(self, voca_num, embedding_dim, filter_lengths=[2,3,4], filter_num=20):
        super(CNNRegressor, self).__init__()
        
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.filter_lengths = filter_lengths
        self.filter_num = filter_num
        
        self.embedding=nn.Embedding(self.voca_num,self.embedding_dim,padding_idx=0)
        self.filters= nn.ModuleList( [nn.Conv1d(1,self.filter_num,(i,embedding_dim)) for i in filter_lengths] )
        
        self.linear1=nn.Linear(len(self.filter_lengths)*self.filter_num,10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self,words):
        embs=self.embedding(words)
        embs=embs.unsqueeze(1)
        
        out=[torch.relu(conv(embs)) for conv in self.filters]
        out=[o.squeeze(3) for o in out] #squeeze를 통해서 마지막에 추가가 되어있는 1을 제거
        out=[F.max_pool1d(feature,feature.size(2)) for feature in out]
        out=torch.cat(out,dim=1)
        out=out.squeeze(2)
        
        out=torch.relu(self.linear1(out))
        out=self.linear2(out)
        
        return out

In [0]:
device=torch.device('cuda')
regressor=CNNRegressor(len(index2voca),128)
regressor=regressor.to(device)

mse_loss=nn.MSELoss()

optimizer=torch.optim.Adam(regressor.parameters(),lr=0.01)

In [76]:
epochs=5

regressor.train()
for e in range(epochs):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y =  batch_X.to(device), batch_y.to(device)
        
        predict=regressor(batch_X)
        
        loss=mse_loss(predict,batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            loss=loss.item()
            print(f"{e}epochs, {i}iters-{loss}")

0epochs, 0iters-0.262140691280365
0epochs, 100iters-0.0741741731762886
0epochs, 200iters-0.07131094485521317
0epochs, 300iters-0.045048102736473083
0epochs, 400iters-0.05284129083156586
0epochs, 500iters-0.06375740468502045
0epochs, 600iters-0.06090252473950386
0epochs, 700iters-0.06008525192737579
0epochs, 800iters-0.05268924683332443
0epochs, 900iters-0.049431148916482925
0epochs, 1000iters-0.051765553653240204
0epochs, 1100iters-0.052678704261779785
0epochs, 1200iters-0.05286268889904022
0epochs, 1300iters-0.05529302358627319
0epochs, 1400iters-0.054035484790802
0epochs, 1500iters-0.04556132107973099
0epochs, 1600iters-0.05497920513153076
0epochs, 1700iters-0.042695675045251846
0epochs, 1800iters-0.06429576873779297
0epochs, 1900iters-0.04851677268743515
1epochs, 0iters-0.04458268731832504
1epochs, 100iters-0.04107225686311722
1epochs, 200iters-0.04824111983180046
1epochs, 300iters-0.03224591910839081
1epochs, 400iters-0.04384275898337364
1epochs, 500iters-0.04912794381380081
1epoch

KeyboardInterrupt: ignored

In [0]:
#원본

class CNNRegressor(nn.Module):
    def __init__(self, voca_num, embedding_dim, filter_lengths, filter_num=20):
        super(CNNRegressor, self).__init__()
        
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.filter_lengths = filter_lengths
        self.filter_num = filter_num
        
        self.embedding = nn.Embedding(self.voca_num, self.embedding_dim, padding_idx=0)
        self.filters = nn.ModuleList([nn.Conv1d(1, self.filter_num, (l, self.embedding_dim)) for l in self.filter_lengths])
        
        self.linear1 = nn.Linear(self.filter_num*len(self.filter_lengths), 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, words):
        embs = self.embedding(words)
        embs = embs.unsqueeze(1)
        features = [F.relu(conv(embs)) for conv in self.filters]
        features = [f.squeeze(3) for f in features]
        features = [F.max_pool1d(f, f.size(2)) for f in features]
        output = torch.cat(features, dim=1)
        output = output.squeeze(2)
        output = torch.relu(self.linear1(output))
        output = self.linear2(output)
        return output
    

class RNNRegressor(nn.Module):
    def __init__(self, voca_num, embedding_dim, hidden_dim, num_layer=2, bidirectional=True):
        super(RNNRegressor, self).__init__()
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(voca_num, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=self.num_layer,
                          bidirectional=self.bidirectional, batch_first=True)
        self.linear1 = nn.Linear(hidden_dim*(self.bidirectional + 1), 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, words):
        embs = self.embedding(words)
        output, h_n = self.gru(embs)
        output = output[:, -1, :]
        output = torch.relu(self.linear1(output))
        output = self.linear2(output)
        return output

In [0]:
device = torch.device('cuda')

# reg = CNNRegressor(len(index2voca), 128, [2, 3, 4])
reg = RNNRegressor(len(index2voca), 256, 256, num_layer=2, bidirectional=True)
reg = reg.to(device)

criterion = nn.MSELoss()
optim = torch.optim.Adam(reg.parameters(), lr=1e-3, weight_decay=1e-6)

In [0]:
epochs = 10

reg.train()
for e in range(epochs):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        predict = reg(batch_X)
        
        loss = criterion(predict, batch_y)
        optim.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(reg.parameters(), 0.5)
        optim.step()
        
        if i % 100 == 0:
            loss = loss.item()
            print(f"{e}epochs, {i}iters - {loss}")

In [0]:
total_loss = []
test_num = 0
l1_loss = nn.L1Loss()

with torch.no_grad():
    reg.eval()
    for i, (batch_X, batch_y) in enumerate(test_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        predict = reg(batch_X)
        predict = torch.clamp(predict, min=0, max=1)
        
        loss = l1_loss(predict, batch_y)
        loss = loss.item()
        batch_size = batch_X.size(0)
        test_num += batch_size
        total_loss.append(loss*batch_size)
        
total_loss = np.sum(total_loss)/test_num
print(total_loss)

0.2779954965793842


In [0]:
from konlpy.tag import Twitter
from glob import glob
from collections import Counter
from collections import defaultdict
from random import shuffle
from pprint import pprint
from scipy.sparse import dok_matrix
import numpy as np
# import ujson as json
import pickle

In [0]:
import numpy as np

In [0]:
sentence = "이 영화 정말 재밌다"
okt=Okt()
parsed = okt.morphs(sentence)

vector=np.zeros((1,100))

for i, word in enumerate(parsed):
    vector[0,i] = voca2index[word]
    
vector = torch.from_numpy(vector).long().to(device)

In [88]:
vector

tensor([[  2,   3,  47, 325,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0]], device='cuda:0')

In [89]:
sentence = "이 영화 정말 재미없다" #해당되는 재미없다 단어가 없는 상태임
okt=Okt()
parsed = okt.morphs(sentence)

vector=np.zeros((1,100))

for i, word in enumerate(parsed):
    vector[0,i] = voca2index[word]
    
vector = torch.from_numpy(vector).long().to(device)

KeyError: ignored

In [0]:
sentence = "이 영화 정말 재미없다" #해당되는 재미없다 단어가 없는 상태임
okt=Okt()
parsed = okt.morphs(sentence)

vector=np.zeros((1,100))

for i, word in enumerate(parsed):
    if word in voca2index:
        vector[0,i] += voca2index[word]
    else:
        vector[0,i] += voca2index['<UNK>']
    
vector = torch.from_numpy(vector).long().to(device)

In [91]:
vector

tensor([[    2,     3,    47, 19999,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')

In [93]:
regressor(vector).item()

0.3064884543418884

In [0]:
sentence = "이 영화 정말 지루하다" 
okt=Okt()
parsed = okt.morphs(sentence)

vector=np.zeros((1,100))

for i, word in enumerate(parsed):
    if word in voca2index:
        vector[0,i] += voca2index[word]
    else:
        vector[0,i] += voca2index['<UNK>']
    
vector = torch.from_numpy(vector).long().to(device)

In [95]:
regressor(vector).item()

0.2387334555387497

In [0]:
class RNNRegressor(nn.Module):
    def __init__(self,voca_num,embedding_dim, hidden_dim):
        super(RNNRegressor,self).__init__()
        
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(voca_num, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, batch_first=True, bidirectional=True,num_layers=2)
        self.linear1 = nn.Linear(2*self.hidden_dim, 10)
        self.linear2 = nn.Linear(10, 1)
        
        
    def forward(self,word_indices):
        embs=self.embedding(word_indices)
        
        output, (h_n, c_n) = self.lstm(embs)
        output= output[:,-1,:]
        
        output=torch.relu(self.linear1(output))
        output=self.linear2(output)
        return output

In [0]:
device = torch.device('cuda')

# reg = CNNRegressor(len(index2voca), 128, [2, 3, 4])
reg = RNNRegressor(len(index2voca), 128, 128)
reg = reg.to(device)

criterion = nn.MSELoss()
optim = torch.optim.Adam(reg.parameters(), lr=1e-3, weight_decay=1e-6)

In [102]:
epochs=5

regressor.train()
for e in range(epochs):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y =  batch_X.to(device), batch_y.to(device)
        
        predict=regressor(batch_X)
        
        loss=mse_loss(predict,batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            loss=loss.item()
            print(f"{e}epochs, {i}iters-{loss}")

0epochs, 0iters-0.022421497851610184
0epochs, 100iters-0.041388075798749924
0epochs, 200iters-0.027331089600920677
0epochs, 300iters-0.05903324484825134
0epochs, 400iters-0.023417159914970398
0epochs, 500iters-0.041844312101602554
0epochs, 600iters-0.0280019361525774
0epochs, 700iters-0.031860027462244034
0epochs, 800iters-0.04632902145385742
0epochs, 900iters-0.023634890094399452
0epochs, 1000iters-0.026830220595002174
0epochs, 1100iters-0.030313536524772644
0epochs, 1200iters-0.033939510583877563
0epochs, 1300iters-0.041720107197761536
0epochs, 1400iters-0.0329010970890522
0epochs, 1500iters-0.036199554800987244
0epochs, 1600iters-0.025153741240501404
0epochs, 1700iters-0.023320890963077545
0epochs, 1800iters-0.04114970564842224
0epochs, 1900iters-0.03554493561387062
1epochs, 0iters-0.026884762570261955
1epochs, 100iters-0.018672293052077293
1epochs, 200iters-0.03155139088630676
1epochs, 300iters-0.022098306566476822
1epochs, 400iters-0.02218051068484783
1epochs, 500iters-0.044945895

KeyboardInterrupt: ignored

rnn 은 dropout을 잘 사용 안함. 정보 손실이 너무 큼

In [0]:
#원본

ext = "이 영화 정말 감동적이다"
parsed = okt.morphs(text)

vector = np.zeros((1, train_X.shape[1]))
for i, w in enumerate(parsed):
    if w in voca2index:
        vector[0, i] += voca2index[w]
    
vector = torch.from_numpy(vector).long().to(device)
predict = torch.sigmoid(reg(vector)).item()
print(predict)

0.6258990168571472
